In [2]:
from __future__ import division 
import h5py
import struct
import ctypes
import pymorton
from numpy import array

ImportError: No module named pymorton

In [ ]:
def getblob(x, y, z, blobsize, h5file):
    #definition used to grab a blob in xyz order.
    u = h5file["U"][x:x+blobsize,y:y+blobsize,z:z+blobsize]
    v = h5file["V"][x:x+blobsize,y:y+blobsize,z:z+blobsize]
    w = h5file["W"][x:x+blobsize,y:y+blobsize,z:z+blobsize]
    buffer = ctypes.create_string_buffer(6144) #hardcoded for speed
    index = 0
    offset = 0
    for k in range(0, blobsize):
        for j in range(0, blobsize):
            for i in range (0, blobsize):
                struct.pack_into('f'*3, buffer, offset,u[k,j,i],v[k,j,i],w[k,j,i])
                offset = offset + 12
    return buffer
#Test it out!

In [ ]:


h5file = h5py.File("outpen/OUTPEN_04.h5") #Open for read only
blobsize = 8
#blob = getblob(0,0,0,blobsize, h5file)

f = open('iso4096-102.bin', 'wb+')
zblob = blobsize * blobsize * blobsize
zmax = pymorton.interleave3(1023,511,511)
zmin = pymorton.interleave3(512,0,0)
for zindex in range(zmin,zmax, zblob):
    print ("zindex = " + str(zindex))
    coord = pymorton.deinterleave3(zindex)
    blob = getblob(coord[2], coord[1], coord[0], blobsize, h5file)
    f.write(blob)
    
f.close()

In [ ]:
N = 1024
corner = np.array([N//2])